# Image Curation in NeMo Curator

In the following notebook, we'll be exploring all of the functionality that NeMo Curator has for image dataset curation.
NeMo Curator has a few built-in modules for 

First, we'll need to install NeMo Curator!

NOTE: Please ensure you meet the [requirements](https://github.com/NVIDIA/NeMo-Curator/tree/main?tab=readme-ov-file#install-nemo-curator) before proceeding!

## Download a Sample Dataset
If you already have a dataset in webdataset format, great! You can skip to the [next section](#create-clip-embeddings).
In order to have a sample dataset to play with, we are going to download a subset of the [Microsoft Common Objects in Context (mscoco)](https://cocodataset.org/#home) dataset.
MSCOCO is a dataset of 600,000 image-text pairs (around 76GB) that takes around 20 minutes to download.
For the sake of this tutorial, we are only going to download a subset of the dataset.
We will download 20,000 image-text pairs (around 3GB).

To download the dataset, we are going to use a tool called img2dataset. Let's install it and download the dataset.

In [ ]:
!pip install img2dataset

First, we need to get a list of URLs that identify where all the images are hosted

In [ ]:
!wget https://huggingface.co/datasets/ChristophSchuhmann/MS_COCO_2017_URL_TEXT/resolve/main/mscoco.parquet

We truncate this list of URLs so we don't download the whole dataset.

In [ ]:
import pandas as pd

NUM_URLS = 20_000
urls = pd.read_parquet("mscoco.parquet")
truncated_urls = urls[:NUM_URLS]
truncated_urls.to_parquet("truncated_mscoco.parquet")

Now, let's start the download.

In [ ]:
!img2dataset \
    --url_list truncated_mscoco.parquet \
    --input_format "parquet" \
    --output_folder mscoco \
    --output_format webdataset \
    --url_col "URL" \
    --caption_col "TEXT" \
    --processes_count 16 \
    --thread_count 64 \
    --resize_mode no

In [ ]:
import os

os._exit(0) # Shut down all kernels to prevent dependencies from conflicting

## Install NeMo Curator

In [ ]:
!pip install cython ipywidgets
# Install from source by default
!pip install --extra-index-url https://pypi.nvidia.com ../../[image]
# !pip install --extra-index-url https://pypi.nvidia.com nemo-curator[image]
%env DASK_DATAFRAME__QUERY_PLANNING False

## Create CLIP Embeddings

### Load the Dataset
Instead of operating on the images directly, most features in NeMo Curator take embeddings as inputs. So, as the first stage in the pipeline, we are going to generate embeddings for all the images in the dataset. To begin, let's load the dataset using NeMo Curator.

In [ ]:
from nemo_curator import get_client

client = get_client(cluster_type="gpu")

In [ ]:
# Change the dataset path if you have your own dataset
dataset_path = "./mscoco"
# Change the unique identifier depending on your dataset
id_col = "key"

In [ ]:
from nemo_curator.datasets import ImageTextPairDataset

dataset = ImageTextPairDataset.from_webdataset(dataset_path, id_col)
# Filter out any entries that failed to download
dataset.metadata = dataset.metadata[dataset.metadata["error_message"].isna()]

### Choose the Embedder
We can now define the embedding creation step in our pipeline. NeMo Curator has support for all [timm](https://pypi.org/project/timm/) models. NeMo Curator's aesthetic classifier is trained on embeddings from `vit_large_patch14_clip_quickgelu_224.openai`, so we will use that.

The cell below will do the following:
1. Download the model `vit_large_patch14_clip_quickgelu_224.openai`.
1. Automatically convert the image preprocessing transformations of `vit_large_patch14_clip_quickgelu_224.openai` from their PyTorch form to DALI.

In [ ]:
from nemo_curator.image.embedders import TimmImageEmbedder

embedding_model = TimmImageEmbedder(
                    "vit_large_patch14_clip_quickgelu_224.openai",
                    pretrained=True,
                    batch_size=1024,
                    num_threads_per_worker=16,
                    normalize_embeddings=True,
                    autocast=False,
                )

We can now create embeddings for the whole dataset. It's important to understand what is going on internally in NeMo Curator so you can modify parameters appropriately.

Once the computation is triggered, the cell below will
1. Load a shard of metadata (a `.parquet` file) onto each GPU you have available using Dask-cuDF.
1. Load a copy of `vit_large_patch14_clip_quickgelu_224.openai` onto each GPU.
1. Repeatedly load images into batches of size `batch_size` onto each GPU with a given threads per worker (`num_threads_per_worker`) using DALI.
1. The model is run on the batch (without `torch.autocast()` since `autocast=False`).
1. The output embeddings of the model are normalized since `normalize_embeddings=True`.


Since NeMo Curator uses Dask, the cell below will not cause the embeddings to be created. The computation will only begin once we inspect the output in the `dataset.metadata.head()` call or when we write to disk using `dataset.save_metadata()` or `dataset.to_webdataset()`.

In [ ]:
# Dask is lazy, so this will not compute embeddings
dataset = embedding_model(dataset)

Since embedding creation can take a long time, we save the embeddings right after they are generated. `dataset.save_metadata()` will add a new column for the image embeddings in the existing `.parquet` files in the dataset directory.

In [ ]:
# This triggers the computation
# You don't need to save and load normally, but we do it here so
# the embeddings aren't recomputed every time they are needed.
dataset.save_metadata()
dataset = ImageTextPairDataset.from_webdataset(dataset_path, id_col)
dataset.metadata.head()

## Aesthetic Classifier
With the embeddings now created, we can use the aesthetic classifier. This classifier assigns a score from 0 to 10 that corresponds to how aesthetically pleasing the image is. A score of 0 means that the image is not pleasant to look at, while a score of 10 is pleasant to look at. The exact classifier used is the LAION-Aesthetics_Predictor V2. More information on the model can be found here: https://laion.ai/blog/laion-aesthetics/.

The following cell will download the model to your local storage at `NEMO_CURATOR_HOME` (`/home/user/.nemo_curator`). The model is only 3.6MB.

In [ ]:
from nemo_curator.image.classifiers import AestheticClassifier

aesthetic_dataset_path = "./aesthetic_dataset"
aesthetic_classifier = AestheticClassifier()

In [ ]:
dataset = aesthetic_classifier(dataset)

After annotating with the aesthetic score, we can filter by the aesthetic score. Finally, we save the resulting filter dataset to a new webdataset.

Unlike `dataset.save_metadata()`, `dataset.to_webdataset()` will modify the tar files as well as the `.parquet` files. It will cause the tar files to be resharded and any entries that have an aesthetic score less than or equal to 6 will be removed.

In [ ]:
dataset.metadata["passes_aesthetic_check"] = dataset.metadata["aesthetic_score"] > 6
dataset.to_webdataset(aesthetic_dataset_path, filter_column="passes_aesthetic_check")

### Visualize Results
Now that we have filtered our dataset based on aesthetic score, we can see what kinds of images are remaining.

In [ ]:
import tarfile
import io
from PIL import Image
import matplotlib.pyplot as plt
import os

def display_image_from_tar(tar_file_path, image_file_name):
    # Open the tar file
    with tarfile.open(tar_file_path, 'r') as tar:
        # Extract the specified image file
        image_file = tar.extractfile(image_file_name)
        
        if image_file is not None:
            # Read the image data
            image_data = image_file.read()
            
            # Create a PIL Image object from the image data
            image = Image.open(io.BytesIO(image_data))
            
            # Display the image using matplotlib
            plt.figure(figsize=(10, 8))
            plt.imshow(image)
            plt.axis('off')  # Hide axes
            plt.title(f"Image: {image_file_name}")
            plt.show()
        else:
            print(f"Image file '{image_file_name}' not found in the tar archive.")
    

output_shard = os.path.join(aesthetic_dataset_path, "00000.tar")
image_file_name = '000000003.jpg'
display_image_from_tar(output_shard, image_file_name)

## Semantic Deduplication

NeMo Curator provides an easy module for semantically deduplicating images. Semantic duplicates are images that contain almost the same information content, but are perceptually different. Two imges of the same dog taken from slightly different angles would be considered semantic duplicates. NeMo Curator' semantic deduplication approach is based on the paper [SemDeDup: Data-efficient learning at web-scale through semantic deduplication](https://arxiv.org/pdf/2303.09540) by Abbas et al which has demonstrated that deduplicating your data can lead to the same downstream performance in *half* the number of training iterations. For more information on the algorithm, you can check out the [documentation page](https://docs.nvidia.com/nemo-framework/user-guide/latest/datacuration/semdedup.html#data-curator-semdedup).

In [ ]:
import os
from nemo_curator.datasets import DocumentDataset
from nemo_curator import ClusteringModel, SemanticClusterLevelDedup

# Convert the dataset
embeddings_dataset = DocumentDataset(dataset.metadata)

semantic_dedup_outputs = "./semantic_deduplication"
os.makedirs(semantic_dedup_outputs, exist_ok=True)

# Run clustering
clustering_output = os.path.join(semantic_dedup_outputs, "cluster_output")
clustering_model = ClusteringModel(
    id_col=id_col,
    embedding_col="image_embedding",
    max_iter=10,
    n_clusters=1,
    clustering_output_dir=clustering_output,
)
clustered_dataset = clustering_model(embeddings_dataset)

In [ ]:
# Run cluster-level dedup
emb_by_cluster_output = os.path.join(clustering_output, "embs_by_nearest_center")
sorted_cluster_output = os.path.join(clustering_output, "sorted")
duplicate_output = os.path.join(semantic_dedup_outputs, "duplicates")

semantic_dedup = SemanticClusterLevelDedup(
    n_clusters=1,
    emb_by_clust_dir=emb_by_cluster_output,
    sorted_clusters_dir=sorted_cluster_output,
    id_col=id_col,
    id_col_type="str",
    embedding_col="image_embedding",
    which_to_keep="hard",
    output_dir=duplicate_output,
)
semantic_dedup.compute_semantic_match_dfs([0.01, 0.001])
deduplicated_dataset_ids = semantic_dedup.extract_dedup_data(eps_to_extract=0.01)

### Remove Duplicates

We got a list of deduplicated image ids. Now we can remove entries with those ids from our dataset, and reshard the dataset to remove them from the tar files.

In [ ]:
# Remove duplicates
deduplicated_dataset_path = "./deduplicated_dataset"
dataset.metadata["is_unique"] = dataset.metadata["key"].isin(deduplicated_dataset_ids.df["key"].compute())
dataset.to_webdataset(deduplicated_dataset_path, "is_unique")

### Visualize Duplicates

To better understand what we did in semantic deduplication, let's examine the output of an intermediate step and visualize some of the duplicates NeMo Curator removed.

In [ ]:
import pandas as pd
import os

cluster_path = os.path.join(duplicate_output, "semdedup_pruning_tables", "cluster_0.parquet")
df = pd.read_parquet(cluster_path)
df = df[~df["eps=0.001"]]
df = df.sort_values("cosine_sim_score", ascending=False)
df.head()

Let's examine what the columns mean.
1. The `id` column represents the id of the datapoint.
1. `max_id` represents the id of the image that `id` is the most similar to.
1. `cosine_sim_score` is the cosine similarity, where `1` indicates the two images are exactly the same and `0` means the images are completely different.
1. `eps=0.01` is `True` if `cosine_sim_score` is `>= 0.99`
1. `eps=0.001` is `True` if `cosine_sim_score` is `>= 0.999`

This dataset has a lot of exact duplicates, so those are caught by `eps=0.001`, and we have filtered out the ids so we don't see them here. There are still a few documents that are similar, but not exact duplicates. We can examine `000008687.jpg` and `000008672.jpg` using our helper function from earlier to see what they look like.

In [ ]:
input_shard = os.path.join(dataset_path, "00000.tar")

display_image_from_tar(input_shard, "000008687.jpg")

In [ ]:
display_image_from_tar(input_shard, "000008672.jpg")

Here, we can see that the images are different. Notably, the baby's head is looking in a different direction. Despite the small differences, these images would still be considered semantic duplicates. Feel free to adjust the epsilon threshold and see what kinds of 